# 1) Read in Files

In [1]:
%run Imports.ipynb
name = 'Kred'

key_featsubgroups = pd.read_pickle('../pickle/2_FS/' + name + '/key_featsubgroups.pkl')
df = pd.read_pickle('../pickle/2_FS/' + name + '/2_df_new_.pkl')

/home/RDC/fliegebe/.conda/envs/Work/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(df[target].value_counts()/df.shape[0])
print('df_shape: ', df.shape)

arrears
1   0.646
0   0.354
Name: count, dtype: float64
df_shape:  (129457, 418)


In [3]:
print(target)

arrears


In [4]:
# from Ben
import sys
import os

# Get the parent directory and add it to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

#from Functions_ANOR_Ben import feature_importance

# 2) Create Model prediction functions

## 2.1) Split dataset into train/testing while excluding demographic features

In [5]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

## 2.2) Model training and prediction

In [6]:
x_train, x_test, y_train, y_test = split_data_4(df)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)


# 3) Run Default Random Forest

In [7]:
def run_basic_RF(df, n_estimators):
    """
    Trains a basic Random Forest model and evaluates its performance.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    n_estimators (int): Number of trees in the Random Forest.

    Returns:
    dict: A dictionary containing evaluation metrics for the trained model.
    """

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = split_data_4(df)

    # Initialize a results dictionary
    results_dict = {}
    # Initialize Random Forest classifier
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

    # Train and evaluate the model
    results_dict = model_pred(X_train, X_test, y_train, y_test, rf, 'rf0', 'opt', results_dict)

    return results_dict


In [ ]:
results_dict = run_basic_RF(df, 100)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)

Starting model training and evaluation...


In [ ]:
results_dict

{'rf0': {'classifier': RandomForestClassifier(random_state=42),
  'cfm_train': array([[36755,     0],
         [    0, 66810]]),
  'cfm_test': array([[ 2682,  6452],
         [ 1631, 15127]]),
  'train_accuracy': 1.0,
  'test_accuracy': 0.6878186312374479,
  'train F1-score label 1': 1.0,
  'train F1-score label 0': 1.0,
  'test F1-score label 1': 0.7891592978062968,
  'test F1-score label 0': 0.39889938276195436,
  'test roc auc score': np.float64(0.7036737409018287),
  'best_params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'monotonic_cst': None,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False},
  'time_m': 1.809039008617401}}

In [ ]:
results_dict['rf0']['best_params']

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

# 4) Run Default XGBoost - with CV

In [ ]:
def run_basic_XGB(df, results_dict):
    """
    Trains an XGBoost classifier using default hyperparameters and evaluates its performance.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    results_dict (dict): Dictionary to store evaluation metrics.

    Returns:
    tuple: A dictionary of default XGBoost hyperparameters and the updated results dictionary.
    """

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = split_data_4(df)

    # Initialize XGBoost classifier with default parameters
    xgbc0 = xgb.XGBClassifier(random_state=42)
    xgbc0.fit(X_train, y_train)  # Fit model to extract default parameters

    # Feature Importance made by Ben
    _ = feature_importance(model=xgbc0, X_train=X_train, y_train=y_train, model_name='XGBoost0')

    # Extract default hyperparameters from the trained model
    default_params = {key: [value] for key, value in xgbc0.get_params().items()}  # Wrap in lists for GridSearchCV

    # Perform GridSearchCV using default parameters (no real tuning, only cross-validation)
    clf0 = GridSearchCV(
        estimator=xgbc0,
        param_grid=default_params,  # Single-value hyperparameters
        scoring='accuracy',
        return_train_score=True,
        verbose=1,
        cv=3  # 3-fold cross-validation
    )

    # Train & evaluate the model using model_pred function
    results_dict = model_pred(X_train, X_test, y_train, y_test, clf0, 'xgbc0', 'search', results_dict)

    return default_params, results_dict


In [ ]:
default_params, results_dict = run_basic_XGB(df, results_dict)
# idea: default params are for the monedo_4

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)

Starting model training and evaluation...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Total execution time: 0.43 minutes


In [ ]:
results_dict

{'rf0': {'classifier': RandomForestClassifier(random_state=42),
  'cfm_train': array([[36755,     0],
         [    0, 66810]]),
  'cfm_test': array([[ 2682,  6452],
         [ 1631, 15127]]),
  'train_accuracy': 1.0,
  'test_accuracy': 0.6878186312374479,
  'train F1-score label 1': 1.0,
  'train F1-score label 0': 1.0,
  'test F1-score label 1': 0.7891592978062968,
  'test F1-score label 0': 0.39889938276195436,
  'test roc auc score': np.float64(0.7036737409018287),
  'best_params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'class_weight': None,
   'criterion': 'gini',
   'max_depth': None,
   'max_features': 'sqrt',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'monotonic_cst': None,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': 42,
   'verbose': 0,
   'warm_start': False},
  'time_m': 1.809039008617401},
 'xgbc0'

# 7) Save results

In [ ]:
with open('../outputs/3_AggModel/' + name + '/model_results.pickle', "wb") as file:
    pickle.dump(results_dict, file, pickle.HIGHEST_PROTOCOL)

results_dict = pickle.load(open('../outputs/3_AggModel/' + name + '/model_results.pickle', 'rb'))

In [ ]:
results_dict['xgbc0']

{'classifier': GridSearchCV(cv=3,
              estimator=XGBClassifier(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False, eval_metric=None,
                                      feature_types=None, feature_weights=None,
                                      gamma=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=Non...
                          'learning_rate': [None], 'max_bin': [None],
                          'max_cat_threshold': [None],
                          'max_cat_to_onehot': [None], 'max_delta_step': [None],
                          'max_depth': [None], 'max_l

In [ ]:
# save_path_xgbc0 = '../pickle/3_Model/results_dict_updated_pre-Bayes.pkl'
# with open(save_path_xgbc0, "wb") as file:
#     pickle.dump(results_dict, file, pickle.HIGHEST_PROTOCOL)

## Saving default params

In [ ]:
default_params

{'objective': ['binary:logistic'],
 'base_score': [None],
 'booster': [None],
 'callbacks': [None],
 'colsample_bylevel': [None],
 'colsample_bynode': [None],
 'colsample_bytree': [None],
 'device': [None],
 'early_stopping_rounds': [None],
 'enable_categorical': [False],
 'eval_metric': [None],
 'feature_types': [None],
 'feature_weights': [None],
 'gamma': [None],
 'grow_policy': [None],
 'importance_type': [None],
 'interaction_constraints': [None],
 'learning_rate': [None],
 'max_bin': [None],
 'max_cat_threshold': [None],
 'max_cat_to_onehot': [None],
 'max_delta_step': [None],
 'max_depth': [None],
 'max_leaves': [None],
 'min_child_weight': [None],
 'missing': [nan],
 'monotone_constraints': [None],
 'multi_strategy': [None],
 'n_estimators': [None],
 'n_jobs': [None],
 'num_parallel_tree': [None],
 'random_state': [42],
 'reg_alpha': [None],
 'reg_lambda': [None],
 'sampling_method': [None],
 'scale_pos_weight': [None],
 'subsample': [None],
 'tree_method': [None],
 'validate_p

In [ ]:
# Ben
save_default_params_path = '../pickle/3_Model/xgb_default_params.pkl'

In [ ]:
# with open(save_default_params_path, 'wb') as f:
#     pickle.dump(default_params, f)

# 8) Compile results: AUC, Accuracy and Time

In [ ]:
#printing benchmark, iterative grid search and randomized search ROC AUC / accuracy scores (test data set)
print(f"Benchmark RandomForest - AUC/accuracy score: {np.round(results_dict['rf0']['test roc auc score'],4)} / {np.round(results_dict['rf0']['test_accuracy'],4)} @ {np.round(results_dict['rf0']['time_m'],2)} minutes")
print(f"Benchmark XGBoost - AUC/accuracy score: {np.round(results_dict['xgbc0']['test roc auc score'],4)} / {np.round(results_dict['xgbc0']['test_accuracy'],4)} @ {np.round(results_dict['xgbc0']['time_m'],2)} minutes")

Benchmark RandomForest - AUC/accuracy score: 0.7037 / 0.6878 @ 1.81 minutes
Benchmark XGBoost - AUC/accuracy score: 0.723 / 0.6962 @ 0.43 minutes
